In [1]:
import pandas as pd
import numpy as np

In [2]:
salary_data = pd.read_excel("salaire.xls", skiprows=5)
immigration_data = pd.read_excel("immigration.xls", skiprows=10)
population_data = pd.read_excel("population.xls", skiprows=5)

### First, as usual, we convert unicode to string.

In [3]:
import unicodedata

def unicode_to_string(word):
    if pd.isnull(word):
        return 
    else:
        return unicodedata.normalize('NFKD', word).encode('ascii','ignore')
    
def data_to_string(data):
    new_data = data.copy()
    
    col_title_unicode = new_data.columns.values.tolist()
    col_title = map(unicode_to_string,col_title_unicode)
    new_data.columns = col_title
    
    for col in new_data.columns:
        not_nan_index = [not ind for ind in new_data[col].isnull()]
        not_nan_value = new_data[col][not_nan_index]
        if type(not_nan_value.iloc[0]) == unicode: #check the first not-NaN value
            new_data[col] = map(unicode_to_string,new_data[col])
            
    return new_data

In [4]:
salary_data = data_to_string(salary_data)
immigration_data = data_to_string(immigration_data)
population_data = data_to_string(population_data)

### Now we will rename the column headers so that we have at least 1 common column for all the data frames. 

In [5]:
def rename_column(data): 
    new_data = data.copy()
    col_title = new_data.columns.tolist()
    for x in xrange(len(col_title)):
        if col_title[x] == 'CODGEO':
            col_title[x] = 'Code Insee' # this will be the pivot column for merging
        if col_title[x] == 'LIBGEO':
            col_title[x] = 'Libelle de la commune'
    new_data.columns = col_title
    return new_data

In [6]:
salary_data = rename_column(salary_data)
immigration_data = rename_column(immigration_data)
population_data = rename_column(population_data)

### We still have problem with departemental code of Corse because they are not in numerical form (2A... or 2B...).

In [7]:
def to_digit(string):
    new_string = list(string)[:]
    for x in xrange(len(new_string)): 
        if not new_string[x].isdigit():
            new_string[x] = '0'
    return int(''.join(new_string))

In [8]:
def replace_insee_code(data): #replace 2AXXX or 2BXXX by 20XXX 
        new_data = data.copy()
        insee_list = new_data['Code Insee'].tolist() 
        new_insee_list = [int(code) if code.isdigit() else to_digit(code) for code in insee_list]
        new_data['Code Insee'] = new_insee_list
        return new_data

In [9]:
salary_data = replace_insee_code(salary_data)
immigration_data = replace_insee_code(immigration_data)
population_data = replace_insee_code(population_data)

### There is still another problem: all the numbers in our excel sheet are read in float format. We need to convert them to int for the immigration and population data.

In [10]:
def float_to_int(data):
    new_data = data.copy()    
    for col in new_data.columns:
        not_nan_index = [not ind for ind in new_data[col].isnull()]
        not_nan_value = new_data[col][not_nan_index]
        if type(not_nan_value.iloc[0]) == np.float64: #check the first not-NaN value
            new_data[col] = new_data[col].round()
    return new_data

In [11]:
immigration_data = float_to_int(immigration_data)
population_data = float_to_int(population_data)

### In immigration data, we will add a column which shows us the total number of adult immigrants  in each town

In [12]:
def add_total_immigrant_variable(data): 
    new_data = data.copy()
    variable_list = []
    for variable in data.columns.tolist(): 
        if 'IMMI1' in variable and 'AGE400' not in variable: 
            variable_list.append(variable)
    new_data['Total immigrant'] = new_data[variable_list].sum(axis=1)
    col = new_data.columns.tolist()
    col = col[:2] + col[-1:] + col[2:-1]
    new_data = new_data[col]
    return new_data

In [13]:
immigration_data = add_total_immigrant_variable(immigration_data)

### Now we try to merge the 3 data frames: right_party_vote, immigration_data, salary_data

In [22]:
right_party_vote = pd.read_excel("right_party_vote.xlsx")

In [23]:
immigration_salary = pd.merge(immigration_data, salary_data, on = ['Code Insee','Libelle de la commune'], how = 'outer')

In [24]:
immigration_salary_population = pd.merge(population_data, immigration_salary, on = ['Code Insee','Libelle de la commune'], how = 'outer')

In [25]:
full_data_right = pd.merge(right_party_vote, immigration_salary_population, on = 'Code Insee', how = 'left')

In [26]:
full_data_right

,Code Insee,Voix,% Voix/Ins,% Voix/Exp,% Abs/Ins,REG,DEP,Libelle de la commune,P12_POP,P12_POP0014,...,SNHMHO12,SNHM1812,SNHM2612,SNHM5012,SNHMF1812,SNHMF2612,SNHMF5012,SNHMH1812,SNHMH2612,SNHMH5012
0,1001,107,18.32,33.44,44.69,82,01,L'Abergement-Clemenciat,777,178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1002,32,14.28,25.20,41.52,82,01,L'Abergement-de-Varey,235,44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1004,1096,13.45,29.76,53.34,82,01,Amberieu-en-Bugey,14233,3038,...,10.945976,9.724445,13.042454,15.461354,9.049192,11.330070,12.313855,10.194692,14.088531,17.557177
3,1005,168,14.38,31.22,52.23,82,01,Amberieux-en-Dombes,1642,338,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1006,22,20.56,42.31,49.53,82,01,Ambleon,110,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1007,301,16.90,31.92,44.78,82,01,Ambronay,2437,547,...,11.590314,9.168451,13.082046,14.080739,8.842310,11.542123,11.931994,9.438508,14.185380,15.754087
6,1008,84,14.77,30.43,48.68,82,01,Ambutrix,739,156,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1009,49,18.99,35.25,44.57,82,01,Andert-et-Condon,338,65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1010,99,13.94,31.03,53.94,82,01,Anglefort,1069,259,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1011,38,14.90,29.92,47.84,82,01,Apremont,385,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Export the full data frame

In [27]:
writer = pd.ExcelWriter('full_data_right.xlsx')
full_data_right.to_excel(writer,'Sheet1')
writer.save()

IOError: [Errno 13] Permission denied: 'full_data_right.xlsx'